## Version 1
Code avec PIL

Déplacement de la voiture OK
Détection de collision 
- Classique
- Circle



In [ ]:
import numpy as np
import math
from PIL import Image
from PIL import ImageChops
from PIL import ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class Car:
    def __init__(self, x, y, theta, L=13):
        self.x = x
        self.y = y
        self.theta = theta  # en degrés
        self.L = L  # empattement (distance entre essieux)

        self.car_width = 9
        self.car_height = 17

        # Créer l'image de la voiture (rectangle rouge)
        self.car_img = Image.new('RGBA', (self.car_width, self.car_height), (255, 0, 0, 255))
       


    def apply_control(self, v, phi_deg, dt=1.0):
        """ Appliquer un contrôle cinématique (modèle vélo simple) """
        theta_rad = math.radians(self.theta)
        phi_rad = math.radians(phi_deg)

        # Déplacement en tenant compte que l'axe y est dirigé vers le bas
        self.x = self.x + v * math.cos(theta_rad) * dt
        self.y = self.y - v * math.sin(theta_rad) * dt  # "-" car l'axe y descend

        self.theta += math.degrees((v / self.L) * math.tan(phi_rad) * dt)
        self.theta %= 360  # Normaliser theta dans [0, 360)

        

    def draw_on_map(self, map_img):
        """ Dessiner la voiture sur la carte en plaçant l'essieu arrière au centre pour la rotation """
        
        # 1. Préparer une grande image transparente
        big_w = self.car_width * 3
        big_h = self.car_height * 3
        big_img = Image.new('RGBA', (big_w, big_h), (0, 0, 0, 0))

        # 2. Positionner la voiture avec essieu arrière au centre
        rear_axle_offset_x = self.car_width // 2
        rear_axle_offset_y = 3*(self.car_height // 4)

        paste_x = big_w // 2 - rear_axle_offset_x + 1
        paste_y = big_h // 2 - rear_axle_offset_y + 1

        big_img.paste(self.car_img, (paste_x, paste_y))

        # 3. Rotation (-90° pour aligner l'avant sur l'axe x)
        rotated_car = big_img.rotate(self.theta - 90, expand=True, resample=Image.BICUBIC)

        # 4. Coller sur la carte
        pos_x = int(self.x - rotated_car.width / 2)
        pos_y = int(self.y - rotated_car.height / 2)

        map_with_car = map_img.copy()
        map_with_car.paste(rotated_car, (pos_x, pos_y), rotated_car)

        

        draw = ImageDraw.Draw(map_with_car)
        draw.point((self.x, self.y), fill='blue')



        return map_with_car
    
    def get_center_from_axle(self):
        """
        Retourne la position du centre géométrique de la voiture en (cx, cy),
        à partir de la position de l'essieu arrière (self.x, self.y).
        """
        theta_rad = math.radians(self.theta)

        dx_img = self.car_width * 0.25  # écart horizontal de l'essieu arrière au centre
        dy_img = 0  # pas de déplacement vertical dans l'image

        dx = dx_img * math.cos(theta_rad)
        dy = dx_img * math.sin(theta_rad)

        return self.x + dx, self.y + dy





def Car_simulation():

    # Initialiser la figure
    fig, ax = plt.subplots(figsize=(8, 8))
    plt.axis('off')

    # Afficher la première image
    map_with_car = car.draw_on_map(map_img)
    img_plot = ax.imshow(map_with_car)

    # Définir la séquence de mouvement
    steps = [(2.0, 0.0), (-2.0, 0.0), (-2.0, 0.0), (2.0, 0.0), (2.0, 30), (-2.0, 30.0), (2.0, -30.0), (-2.0, -30.0), (-2.0, 30.0), (2.0, 30.0), (-2.0, -30.0), (2.0, -30.0)]

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à gauche
    for _ in range(35):
        steps.append((2.0, -30.0))  # braquage 30° à gauche

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à gauche
    for _ in range(35):
        steps.append((2.0, 30.0))  # braquage 30° à gauche

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à droite
    #for _ in range(20):
    #   steps.append((-2.0, -30.0))  # braquage -30° à droite

    # Simulation
    for v, phi in steps:
        car.apply_control(v, phi)
        map_with_car = car.draw_on_map(map_img)
        img_plot.set_data(map_with_car)
        plt.pause(0.05)  # Pause courte pour effet animation
        print(car.y)





def is_valid(x, y, theta, car, map_img):
    """
    Retourne True si la voiture placée en (x, y, theta) n'entre pas en collision avec la map.
    """
    # 1. Dessiner la voiture à l'état donné sur une image transparente
    car.x = x
    car.y = y
    car.theta = theta
    car_img_on_map = car.draw_on_map(Image.new('RGBA', map_img.size, (0, 0, 0, 0)))

    # 2. Créer des masques binaires (mode '1')
    car_mask = car_img_on_map.convert('L').point(lambda p: 1 if p > 0 else 0, mode='1')
    map_mask = map_img.convert('L').point(lambda p: 1 if p < 100 else 0, mode='1')  # obstacles sombres

    # 3. Intersection des deux masques
    overlap = ImageChops.logical_and(car_mask, map_mask)

    # 4. Collision si au moins un pixel blanc dans overlap
    if overlap.getbbox():
        return False  # Collision
    return True       # Libre


def show_car_with_circle(x, y, theta, car, map_img):
    """
    Affiche la voiture à (x, y, theta) sur la carte avec son cercle de collision centré au milieu géométrique.
    """
    # Rayon du cercle de collision
    r = math.sqrt((car.car_width/2)**2+(car.car_height/2)**2)

    # Calcul du centre géométrique de la voiture (à mi-longueur)
    theta_rad = math.radians(theta)
    cx, cy = x+int(0.25*car.car_height*math.cos(theta)), y-int(0.25*car.car_height*math.sin(theta))
    

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


def show_car_circle(x, y, theta, car, map_img):
    """
    Affiche la voiture à (x, y, theta) sur la carte avec son cercle de collision centré au milieu géométrique.
    """
    # Rayon du cercle de collision
    

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


def is_valid_circle_ANCIEN(x, y, theta, car, map_img, depth=0, max_depth=2):
    """
    Teste la collision en utilisant une bounding sphere (puis 2 sous-sphères récursives).
    Retourne True si pas de collision, False sinon.
    """

    # 1. Convertir la map en array binaire : 1 = obstacle, 0 = libre
    map_gray = map_img.convert('L')
    map_array = np.array(map_gray)
    binary_map = (map_array < 100).astype(np.uint8)

    height, width = binary_map.shape

    # 2. Rayon de la bounding sphere
    r = math.sqrt((car.car_width/2)**2+(car.car_height/2)**2)

    # 3. Centre du cercle
    cx, cy = x+int(0.25*car.car_height*math.cos(theta)), y-int(0.25*car.car_height*math.sin(theta))

    # 4. Vérification : si le cercle sort de la carte, collision
    if cx < 0 or cy < 0 or cx >= width or cy >= height:
        return False

    # 5. Vérifie les pixels dans le disque de rayon r
    y_min = max(0, int(cy - r))
    y_max = min(height, int(cy + r) + 1)
    x_min = max(0, int(cx - r))
    x_max = min(width, int(cx + r) + 1)

    for yy in range(y_min, y_max):
        for xx in range(x_min, x_max):
            if (xx - cx)**2 + (yy - cy)**2 <= r**2:
                if binary_map[yy, xx] == 1:  # obstacle
                    break
        else:
            continue
        break
    else:
        return True  # pas de collision dans le grand cercle

    # Si collision et profondeur atteinte : STOP
    if depth >= max_depth:
        return False

    # 6. Sinon subdiviser : créer 2 sous-cercles plus petits
    theta_rad = math.radians(theta)

    # Longueur de demi-voiture (axe longitudinal)
    dx = (car.car_height / 4) * math.cos(theta_rad)
    dy = -(car.car_height / 4) * math.sin(theta_rad)  # axe y va vers le bas

    center1 = (x + dx, y + dy)
    center2 = (x - dx, y - dy)

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


    return (is_valid_circle(center1[0], center1[1], theta, car, map_img, depth + 1, max_depth) and
            is_valid_circle(center2[0], center2[1], theta, car, map_img, depth + 1, max_depth))




def is_valid_circle(x, y, theta, car, map_img, depth=0, max_depth=2, visualize=True):
    """
    Teste la collision en utilisant une bounding sphere (puis 2 sous-sphères récursives).
    Retourne True si pas de collision, False sinon.
    Affiche les cercles si visualize=True.
    """

    # 1. Convertir la map en array binaire : 1 = obstacle, 0 = libre
    map_gray = map_img.convert('L')
    map_array = np.array(map_gray)
    binary_map = (map_array < 100).astype(np.uint8)

    height, width = binary_map.shape

    # 2. Rayon de la bounding sphere
    r = math.sqrt((car.car_width / 2) ** 2 + (car.car_height / 2) ** 2)

    # 3. Centre du cercle principal (milieu géométrique de la voiture)
    theta_rad = math.radians(theta)
    cx, cy = x+int(0.25*car.car_height*math.cos(theta))+1, y-int(0.25*car.car_height*math.sin(theta))-1

    # 4. Vérification des bords
    if cx < 0 or cy < 0 or cx >= width or cy >= height:
        return False

    # 5. Test de collision par pixel dans le disque
    y_min = max(0, int(cy - r))
    y_max = min(height, int(cy + r) + 1)
    x_min = max(0, int(cx - r))
    x_max = min(width, int(cx + r) + 1)

    collision = False
    for yy in range(y_min, y_max):
        for xx in range(x_min, x_max):
            if (xx - cx)**2 + (yy - cy)**2 <= r**2:
                if binary_map[int(yy), int(xx)] == 1:
                    collision = True
                    break
        if collision:
            break

    # 6. Affichage
    if visualize:
        car.x = x
        car.y = y
        car.theta = theta
        map_with_car = car.draw_on_map(map_img)

        fig, ax = plt.subplots(figsize=(8, 8))
        ax.imshow(map_with_car)
        ax.set_title(f"Collision check — depth {depth}")
        ax.axis('off')

        color = 'lime' if not collision else 'red'
        ax.add_patch(patches.Circle((cx, cy), radius=r, linewidth=1.5, edgecolor=color, facecolor='none'))
        ax.plot(cx, cy, 'o', color=color, markersize=4)

    if not collision:
        if visualize:
            plt.show()
        return True  # libre, pas de collision

    # Si collision et profondeur max atteinte
    if depth >= max_depth:
        if visualize:
            plt.show()
        return False

    # 7. Subdivision en 2 sous-cercles
    offset = (car.car_height // 4)
    dx = offset * math.cos(theta_rad)
    dy = offset * math.sin(theta_rad)

    center1 = (x, y - 2*dy)
    center2 = (x, y)

    radius = math.sqrt((car.car_width / 2) ** 2 + (car.car_height / 4) ** 2)

    if visualize:
        # Afficher les 2 sous-cercles
        ax.add_patch(patches.Circle(center1, radius=radius, edgecolor='orange', fill=False, linestyle='--'))
        ax.add_patch(patches.Circle(center2, radius=radius, edgecolor='orange', fill=False, linestyle='--'))
        ax.plot(center1[0], center1[1], 'o', color='orange', markersize=3)
        ax.plot(center2[0], center2[1], 'o', color='orange', markersize=3)
        plt.show()

    # 8. Appel récursif
    return (is_valid_circle(center1[0], center1[1], theta, car, map_img, depth + 1, max_depth, visualize) and
            is_valid_circle(center2[0], center2[1], theta, car, map_img, depth + 1, max_depth, visualize))

    














# Charger la carte
map_img = Image.open('mapLittle.png').convert('RGBA')

#Coordonnées de la voiture
x = 150
y = 150
theta = 0


# Créer la voiture
car = Car(x, y, theta)



Car_simulation()




#show_car_with_circle(x, y, theta, car, map_img)
#show_car(x, y, theta, car, map_img, show_circle=False)





#result = is_valid(x, y, theta, car, map_img)
#print("Libre" if result else "Collision")

#result = is_valid_circle(x, y, theta, car, map_img)
#print("Libre" if result else "Collision")

## Version 2

Code avec PIL

Modification pour le centre de rotation soit constament situé au milieu de l'essieu arrière.

Mais voiture qui se déforme.

In [ ]:
import numpy as np
import math
from PIL import Image
from PIL import ImageChops
from PIL import ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import numpy as np
import cv2
import math

class Car:
    def __init__(self, x, y, theta, L=13):
        self.x = x
        self.y = y
        self.theta = theta  # en degrés
        self.L = L  # empattement (distance entre essieux)

        self.car_width = 9
        self.car_height = 17

        # Création d'une image de la voiture autour de l'essieu arrière
        self.rear_axle_offset_y = int(0.75 * self.car_height)
        self.img_height = 2 * self.rear_axle_offset_y
        self.img_width = 2 * (self.car_width // 2)

        # Créer l'image RGBA de la voiture (fond transparent)
        self.car_img = np.zeros((self.img_height, self.img_width, 4), dtype=np.uint8)

        top_left = (self.img_width // 2 - self.car_width // 2,
                    self.rear_axle_offset_y - self.car_height + 1)
        bottom_right = (self.img_width // 2 + self.car_width // 2,
                        self.rear_axle_offset_y + 1)

        # Dessiner la voiture en rouge (avec alpha à 255)
        cv2.rectangle(self.car_img, top_left, bottom_right, (0, 0, 255, 255), thickness=-1)

        # Colorier le centre de rotation en bleu
        center_x = self.img_width // 2
        center_y = self.rear_axle_offset_y
        self.car_img[center_y, center_x] = (255, 0, 0, 255)  # BGR-A format

    def apply_control(self, v, phi_deg, dt=1.0):
        """ Appliquer un contrôle cinématique (modèle vélo simple) """
        theta_rad = math.radians(self.theta)
        phi_rad = math.radians(phi_deg)

        self.x = self.x + v * math.cos(theta_rad) * dt
        self.y = self.y - v * math.sin(theta_rad) * dt  # Axe y vers le bas

        self.theta += math.degrees((v / self.L) * math.tan(phi_rad) * dt)
        self.theta %= 360

    def draw_on_map(self, map_img):
        """ Dessiner la voiture sur la carte avec OpenCV """

        # Rotation de la voiture autour de son centre
        center = (self.car_img.shape[1] // 2, self.car_img.shape[0] // 2)
        rot_mat = cv2.getRotationMatrix2D(center, -(self.theta - 90), 1.0)
        rotated_car = cv2.warpAffine(self.car_img, rot_mat,
                                     (self.car_img.shape[1], self.car_img.shape[0]),
                                     flags=cv2.INTER_LINEAR,
                                     borderMode=cv2.BORDER_CONSTANT,
                                     borderValue=(0, 0, 0, 0))

        # Convertir la carte en 4 canaux si elle n’est pas déjà en BGRA
        if map_img.shape[2] == 3:
            map_img = cv2.cvtColor(map_img, cv2.COLOR_BGR2BGRA)

        # Calcul de la position où dessiner la voiture
        pos_x = int(self.x - rotated_car.shape[1] / 2)
        pos_y = int(self.y - rotated_car.shape[0] / 2)

        # Créer une copie de la carte
        map_with_car = map_img.copy()

        # Zone de destination
        h, w = rotated_car.shape[:2]
        roi = map_with_car[pos_y:pos_y + h, pos_x:pos_x + w]

        # Gérer les cas de débordement de bordure
        if roi.shape[0] != h or roi.shape[1] != w:
            return map_with_car  # Ne rien dessiner si ça dépasse

        # Fusion alpha : img_result = car_alpha * car + (1 - car_alpha) * map
        alpha_car = rotated_car[:, :, 3:] / 255.0
        alpha_map = 1.0 - alpha_car
        for c in range(3):  # B, G, R channels
            roi[:, :, c] = (alpha_car[:, :, 0] * rotated_car[:, :, c] +
                            alpha_map[:, :, 0] * roi[:, :, c]).astype(np.uint8)

        map_with_car[pos_y:pos_y + h, pos_x:pos_x + w] = roi

        return map_with_car


    
    def get_center_from_axle(self):
        """
        Retourne la position du centre géométrique de la voiture en (cx, cy),
        à partir de la position de l'essieu arrière (self.x, self.y).
        """
        theta_rad = math.radians(self.theta)

        dx_img = self.car_width * 0.25  # écart horizontal de l'essieu arrière au centre
        dy_img = 0  # pas de déplacement vertical dans l'image

        dx = dx_img * math.cos(theta_rad)
        dy = dx_img * math.sin(theta_rad)

        return self.x + dx, self.y + dy





def Car_simulation():

    # Initialiser la figure
    fig, ax = plt.subplots(figsize=(8, 8))
    plt.axis('off')

    # Afficher la première image
    map_with_car = car.draw_on_map(map_img)
    img_plot = ax.imshow(map_with_car)

    # Définir la séquence de mouvement
    steps = [(2.0, 0.0), (-2.0, 0.0), (-2.0, 0.0), (2.0, 0.0), (2.0, 30), (-2.0, 30.0), (2.0, -30.0), (-2.0, -30.0), (-2.0, 30.0), (2.0, 30.0), (-2.0, -30.0), (2.0, -30.0)]

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à gauche
    for _ in range(35):
        steps.append((2.0, -30.0))  # braquage 30° à gauche

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à gauche
    for _ in range(35):
        steps.append((2.0, -30.0))  # braquage 30° à gauche

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à droite
    #for _ in range(20):
    #   steps.append((-2.0, -30.0))  # braquage -30° à droite

    # Simulation
    for v, phi in steps:
        car.apply_control(v, phi)
        map_with_car = car.draw_on_map(map_img)
        img_plot.set_data(map_with_car)
        plt.pause(0.05)  # Pause courte pour effet animation
        print(car.y)





def is_valid(x, y, theta, car, map_img):
    """
    Retourne True si la voiture placée en (x, y, theta) n'entre pas en collision avec la map.
    """
    # 1. Dessiner la voiture à l'état donné sur une image transparente
    car.x = x
    car.y = y
    car.theta = theta
    car_img_on_map = car.draw_on_map(Image.new('RGBA', map_img.size, (0, 0, 0, 0)))

    # 2. Créer des masques binaires (mode '1')
    car_mask = car_img_on_map.convert('L').point(lambda p: 1 if p > 0 else 0, mode='1')
    map_mask = map_img.convert('L').point(lambda p: 1 if p < 100 else 0, mode='1')  # obstacles sombres

    # 3. Intersection des deux masques
    overlap = ImageChops.logical_and(car_mask, map_mask)

    # 4. Collision si au moins un pixel blanc dans overlap
    if overlap.getbbox():
        return False  # Collision
    return True       # Libre


def show_car_with_circle(x, y, theta, car, map_img):
    """
    Affiche la voiture à (x, y, theta) sur la carte avec son cercle de collision centré au milieu géométrique.
    """
    # Rayon du cercle de collision
    r = math.sqrt((car.car_width/2)**2+(car.car_height/2)**2)

    # Calcul du centre géométrique de la voiture (à mi-longueur)
    theta_rad = math.radians(theta)
    cx, cy = x+int(0.25*car.car_height*math.cos(theta)), y-int(0.25*car.car_height*math.sin(theta))
    

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


def show_car_circle(x, y, theta, car, map_img):
    """
    Affiche la voiture à (x, y, theta) sur la carte avec son cercle de collision centré au milieu géométrique.
    """
    # Rayon du cercle de collision
    

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


def is_valid_circle_ANCIEN(x, y, theta, car, map_img, depth=0, max_depth=2):
    """
    Teste la collision en utilisant une bounding sphere (puis 2 sous-sphères récursives).
    Retourne True si pas de collision, False sinon.
    """

    # 1. Convertir la map en array binaire : 1 = obstacle, 0 = libre
    map_gray = map_img.convert('L')
    map_array = np.array(map_gray)
    binary_map = (map_array < 100).astype(np.uint8)

    height, width = binary_map.shape

    # 2. Rayon de la bounding sphere
    r = math.sqrt((car.car_width/2)**2+(car.car_height/2)**2)

    # 3. Centre du cercle
    cx, cy = x+int(0.25*car.car_height*math.cos(theta)), y-int(0.25*car.car_height*math.sin(theta))

    # 4. Vérification : si le cercle sort de la carte, collision
    if cx < 0 or cy < 0 or cx >= width or cy >= height:
        return False

    # 5. Vérifie les pixels dans le disque de rayon r
    y_min = max(0, int(cy - r))
    y_max = min(height, int(cy + r) + 1)
    x_min = max(0, int(cx - r))
    x_max = min(width, int(cx + r) + 1)

    for yy in range(y_min, y_max):
        for xx in range(x_min, x_max):
            if (xx - cx)**2 + (yy - cy)**2 <= r**2:
                if binary_map[yy, xx] == 1:  # obstacle
                    break
        else:
            continue
        break
    else:
        return True  # pas de collision dans le grand cercle

    # Si collision et profondeur atteinte : STOP
    if depth >= max_depth:
        return False

    # 6. Sinon subdiviser : créer 2 sous-cercles plus petits
    theta_rad = math.radians(theta)

    # Longueur de demi-voiture (axe longitudinal)
    dx = (car.car_height / 4) * math.cos(theta_rad)
    dy = -(car.car_height / 4) * math.sin(theta_rad)  # axe y va vers le bas

    center1 = (x + dx, y + dy)
    center2 = (x - dx, y - dy)

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


    return (is_valid_circle(center1[0], center1[1], theta, car, map_img, depth + 1, max_depth) and
            is_valid_circle(center2[0], center2[1], theta, car, map_img, depth + 1, max_depth))




def is_valid_circle(x, y, theta, car, map_img, depth=0, max_depth=2, visualize=True):
    """
    Teste la collision en utilisant une bounding sphere (puis 2 sous-sphères récursives).
    Retourne True si pas de collision, False sinon.
    Affiche les cercles si visualize=True.
    """

    # 1. Convertir la map en array binaire : 1 = obstacle, 0 = libre
    map_gray = map_img.convert('L')
    map_array = np.array(map_gray)
    binary_map = (map_array < 100).astype(np.uint8)

    height, width = binary_map.shape

    # 2. Rayon de la bounding sphere
    r = math.sqrt((car.car_width / 2) ** 2 + (car.car_height / 2) ** 2)

    # 3. Centre du cercle principal (milieu géométrique de la voiture)
    theta_rad = math.radians(theta)
    cx, cy = x+int(0.25*car.car_height*math.cos(theta))+1, y-int(0.25*car.car_height*math.sin(theta))-1

    # 4. Vérification des bords
    if cx < 0 or cy < 0 or cx >= width or cy >= height:
        return False

    # 5. Test de collision par pixel dans le disque
    y_min = max(0, int(cy - r))
    y_max = min(height, int(cy + r) + 1)
    x_min = max(0, int(cx - r))
    x_max = min(width, int(cx + r) + 1)

    collision = False
    for yy in range(y_min, y_max):
        for xx in range(x_min, x_max):
            if (xx - cx)**2 + (yy - cy)**2 <= r**2:
                if binary_map[int(yy), int(xx)] == 1:
                    collision = True
                    break
        if collision:
            break

    # 6. Affichage
    if visualize:
        car.x = x
        car.y = y
        car.theta = theta
        map_with_car = car.draw_on_map(map_img)

        fig, ax = plt.subplots(figsize=(8, 8))
        ax.imshow(map_with_car)
        ax.set_title(f"Collision check — depth {depth}")
        ax.axis('off')

        color = 'lime' if not collision else 'red'
        ax.add_patch(patches.Circle((cx, cy), radius=r, linewidth=1.5, edgecolor=color, facecolor='none'))
        ax.plot(cx, cy, 'o', color=color, markersize=4)

    if not collision:
        if visualize:
            plt.show()
        return True  # libre, pas de collision

    # Si collision et profondeur max atteinte
    if depth >= max_depth:
        if visualize:
            plt.show()
        return False

    # 7. Subdivision en 2 sous-cercles
    offset = (car.car_height // 4)
    dx = offset * math.cos(theta_rad)
    dy = offset * math.sin(theta_rad)

    center1 = (x, y - 2*dy)
    center2 = (x, y)

    radius = math.sqrt((car.car_width / 2) ** 2 + (car.car_height / 4) ** 2)

    if visualize:
        # Afficher les 2 sous-cercles
        ax.add_patch(patches.Circle(center1, radius=radius, edgecolor='orange', fill=False, linestyle='--'))
        ax.add_patch(patches.Circle(center2, radius=radius, edgecolor='orange', fill=False, linestyle='--'))
        ax.plot(center1[0], center1[1], 'o', color='orange', markersize=3)
        ax.plot(center2[0], center2[1], 'o', color='orange', markersize=3)
        plt.show()

    # 8. Appel récursif
    return (is_valid_circle(center1[0], center1[1], theta, car, map_img, depth + 1, max_depth, visualize) and
            is_valid_circle(center2[0], center2[1], theta, car, map_img, depth + 1, max_depth, visualize))

    














# Charger la carte
map_img = Image.open('mapHyperLittle.png').convert('RGBA')

#Coordonnées de la voiture
x = 50
y = 50
theta = 0


# Créer la voiture
car = Car(x, y, theta)



Car_simulation()




#show_car_with_circle(x, y, theta, car, map_img)
#show_car(x, y, theta, car, map_img, show_circle=False)





#result = is_valid(x, y, theta, car, map_img)
#print("Libre" if result else "Collision")

#result = is_valid_circle(x, y, theta, car, map_img)
#print("Libre" if result else "Collision")

In [ ]:
import numpy as np
import math
from PIL import Image
from PIL import ImageChops
from PIL import ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class Car:
    def __init__(self, x, y, theta, L=13):
        self.x = x
        self.y = y
        self.theta = theta  # en degrés
        self.L = L  # empattement (distance entre essieux)

        self.car_width = 9
        self.car_height = 17

        # Créer une image centrée autour de l'essieu arrière
        rear_axle_offset_y = int(0.75 * self.car_height)
        img_height = 2 * rear_axle_offset_y
        img_width = 2 * (self.car_width // 2)

        self.car_img = Image.new('RGBA', (img_width, img_height), (0, 0, 0, 0))
        draw = ImageDraw.Draw(self.car_img)
        top_left = (img_width // 2 - self.car_width // 2, rear_axle_offset_y - self.car_height + 1)
        bottom_right = (img_width // 2 + self.car_width // 2, rear_axle_offset_y + 1)
        draw.rectangle([top_left, bottom_right], fill=(255, 0, 0, 255))

        # Colorier le pixel du centre de rotation en bleu (écrase le rouge à cet endroit)
        center_x = img_width // 2
        center_y = rear_axle_offset_y
        self.car_img.putpixel((center_x, center_y), (0, 0, 255, 255))
        


    def apply_control(self, v, phi_deg, dt=1.0):
        """ Appliquer un contrôle cinématique (modèle vélo simple) """
        theta_rad = math.radians(self.theta)
        phi_rad = math.radians(phi_deg)

        # Déplacement en tenant compte que l'axe y est dirigé vers le bas
        self.x = self.x + v * math.cos(theta_rad) * dt
        self.y = self.y - v * math.sin(theta_rad) * dt  # "-" car l'axe y descend

        self.theta += math.degrees((v / self.L) * math.tan(phi_rad) * dt)
        self.theta %= 360  # Normaliser theta dans [0, 360)

        

    def draw_on_map(self, map_img):
        """ Dessiner la voiture sur la carte avec le centre de rotation à l'essieu arrière """

        rotated_car = self.car_img.rotate(self.theta - 90, expand=True, resample=Image.BICUBIC)

        pos_x = int(self.x - rotated_car.width / 2)
        pos_y = int(self.y - rotated_car.height / 2)

        map_with_car = map_img.copy()
        map_with_car.paste(rotated_car, (pos_x, pos_y), rotated_car)

        return map_with_car

    
    def get_center_from_axle(self):
        """
        Retourne la position du centre géométrique de la voiture en (cx, cy),
        à partir de la position de l'essieu arrière (self.x, self.y).
        """
        theta_rad = math.radians(self.theta)

        dx_img = self.car_width * 0.25  # écart horizontal de l'essieu arrière au centre
        dy_img = 0  # pas de déplacement vertical dans l'image

        dx = dx_img * math.cos(theta_rad)
        dy = dx_img * math.sin(theta_rad)

        return self.x + dx, self.y + dy





def Car_simulation():

    # Initialiser la figure
    fig, ax = plt.subplots(figsize=(8, 8))
    plt.axis('off')

    # Afficher la première image
    map_with_car = car.draw_on_map(map_img)
    img_plot = ax.imshow(map_with_car)

    # Définir la séquence de mouvement
    steps = [(2.0, 0.0), (-2.0, 0.0), (-2.0, 0.0), (2.0, 0.0), (2.0, 30), (-2.0, 30.0), (2.0, -30.0), (-2.0, -30.0), (-2.0, 30.0), (2.0, 30.0), (-2.0, -30.0), (2.0, -30.0)]

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à gauche
    for _ in range(35):
        steps.append((2.0, -30.0))  # braquage 30° à gauche

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à gauche
    for _ in range(35):
        steps.append((2.0, -30.0))  # braquage 30° à gauche

    # Avancer tout droit
    for _ in range(30):
        steps.append((2.0, 0.0))  # vitesse 2.0, braquage 0°

    # Tourner à droite
    #for _ in range(20):
    #   steps.append((-2.0, -30.0))  # braquage -30° à droite

    # Simulation
    for v, phi in steps:
        car.apply_control(v, phi)
        map_with_car = car.draw_on_map(map_img)
        img_plot.set_data(map_with_car)
        plt.pause(0.05)  # Pause courte pour effet animation
        print(car.y)





def is_valid(x, y, theta, car, map_img):
    """
    Retourne True si la voiture placée en (x, y, theta) n'entre pas en collision avec la map.
    """
    # 1. Dessiner la voiture à l'état donné sur une image transparente
    car.x = x
    car.y = y
    car.theta = theta
    car_img_on_map = car.draw_on_map(Image.new('RGBA', map_img.size, (0, 0, 0, 0)))

    # 2. Créer des masques binaires (mode '1')
    car_mask = car_img_on_map.convert('L').point(lambda p: 1 if p > 0 else 0, mode='1')
    map_mask = map_img.convert('L').point(lambda p: 1 if p < 100 else 0, mode='1')  # obstacles sombres

    # 3. Intersection des deux masques
    overlap = ImageChops.logical_and(car_mask, map_mask)

    # 4. Collision si au moins un pixel blanc dans overlap
    if overlap.getbbox():
        return False  # Collision
    return True       # Libre


def show_car_with_circle(x, y, theta, car, map_img):
    """
    Affiche la voiture à (x, y, theta) sur la carte avec son cercle de collision centré au milieu géométrique.
    """
    # Rayon du cercle de collision
    r = math.sqrt((car.car_width/2)**2+(car.car_height/2)**2)

    # Calcul du centre géométrique de la voiture (à mi-longueur)
    theta_rad = math.radians(theta)
    cx, cy = x+int(0.25*car.car_height*math.cos(theta)), y-int(0.25*car.car_height*math.sin(theta))
    

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


def show_car_circle(x, y, theta, car, map_img):
    """
    Affiche la voiture à (x, y, theta) sur la carte avec son cercle de collision centré au milieu géométrique.
    """
    # Rayon du cercle de collision
    

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


def is_valid_circle_ANCIEN(x, y, theta, car, map_img, depth=0, max_depth=2):
    """
    Teste la collision en utilisant une bounding sphere (puis 2 sous-sphères récursives).
    Retourne True si pas de collision, False sinon.
    """

    # 1. Convertir la map en array binaire : 1 = obstacle, 0 = libre
    map_gray = map_img.convert('L')
    map_array = np.array(map_gray)
    binary_map = (map_array < 100).astype(np.uint8)

    height, width = binary_map.shape

    # 2. Rayon de la bounding sphere
    r = math.sqrt((car.car_width/2)**2+(car.car_height/2)**2)

    # 3. Centre du cercle
    cx, cy = x+int(0.25*car.car_height*math.cos(theta)), y-int(0.25*car.car_height*math.sin(theta))

    # 4. Vérification : si le cercle sort de la carte, collision
    if cx < 0 or cy < 0 or cx >= width or cy >= height:
        return False

    # 5. Vérifie les pixels dans le disque de rayon r
    y_min = max(0, int(cy - r))
    y_max = min(height, int(cy + r) + 1)
    x_min = max(0, int(cx - r))
    x_max = min(width, int(cx + r) + 1)

    for yy in range(y_min, y_max):
        for xx in range(x_min, x_max):
            if (xx - cx)**2 + (yy - cy)**2 <= r**2:
                if binary_map[yy, xx] == 1:  # obstacle
                    break
        else:
            continue
        break
    else:
        return True  # pas de collision dans le grand cercle

    # Si collision et profondeur atteinte : STOP
    if depth >= max_depth:
        return False

    # 6. Sinon subdiviser : créer 2 sous-cercles plus petits
    theta_rad = math.radians(theta)

    # Longueur de demi-voiture (axe longitudinal)
    dx = (car.car_height / 4) * math.cos(theta_rad)
    dy = -(car.car_height / 4) * math.sin(theta_rad)  # axe y va vers le bas

    center1 = (x + dx, y + dy)
    center2 = (x - dx, y - dy)

    # Dessiner la voiture sur la carte
    
    map_with_car = car.draw_on_map(map_img)

    # Afficher
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(map_with_car)
    ax.set_title(f"Cercle de collision — x={x:.1f}, y={y:.1f}, θ={theta:.1f}°")
    ax.axis('off')

    # Cercle vert centré au bon endroit
    circle = patches.Circle((cx, cy), radius=r, linewidth=1, edgecolor='lime', facecolor='none')
    ax.add_patch(circle)

    # Centre du cercle
    ax.plot(cx, cy, 'go', markersize=5)

    plt.show()


    return (is_valid_circle(center1[0], center1[1], theta, car, map_img, depth + 1, max_depth) and
            is_valid_circle(center2[0], center2[1], theta, car, map_img, depth + 1, max_depth))




def is_valid_circle(x, y, theta, car, map_img, depth=0, max_depth=2, visualize=True):
    """
    Teste la collision en utilisant une bounding sphere (puis 2 sous-sphères récursives).
    Retourne True si pas de collision, False sinon.
    Affiche les cercles si visualize=True.
    """

    # 1. Convertir la map en array binaire : 1 = obstacle, 0 = libre
    map_gray = map_img.convert('L')
    map_array = np.array(map_gray)
    binary_map = (map_array < 100).astype(np.uint8)

    height, width = binary_map.shape

    # 2. Rayon de la bounding sphere
    r = math.sqrt((car.car_width / 2) ** 2 + (car.car_height / 2) ** 2)

    # 3. Centre du cercle principal (milieu géométrique de la voiture)
    theta_rad = math.radians(theta)
    cx, cy = x+int(0.25*car.car_height*math.cos(theta))+1, y-int(0.25*car.car_height*math.sin(theta))-1

    # 4. Vérification des bords
    if cx < 0 or cy < 0 or cx >= width or cy >= height:
        return False

    # 5. Test de collision par pixel dans le disque
    y_min = max(0, int(cy - r))
    y_max = min(height, int(cy + r) + 1)
    x_min = max(0, int(cx - r))
    x_max = min(width, int(cx + r) + 1)

    collision = False
    for yy in range(y_min, y_max):
        for xx in range(x_min, x_max):
            if (xx - cx)**2 + (yy - cy)**2 <= r**2:
                if binary_map[int(yy), int(xx)] == 1:
                    collision = True
                    break
        if collision:
            break

    # 6. Affichage
    if visualize:
        car.x = x
        car.y = y
        car.theta = theta
        map_with_car = car.draw_on_map(map_img)

        fig, ax = plt.subplots(figsize=(8, 8))
        ax.imshow(map_with_car)
        ax.set_title(f"Collision check — depth {depth}")
        ax.axis('off')

        color = 'lime' if not collision else 'red'
        ax.add_patch(patches.Circle((cx, cy), radius=r, linewidth=1.5, edgecolor=color, facecolor='none'))
        ax.plot(cx, cy, 'o', color=color, markersize=4)

    if not collision:
        if visualize:
            plt.show()
        return True  # libre, pas de collision

    # Si collision et profondeur max atteinte
    if depth >= max_depth:
        if visualize:
            plt.show()
        return False

    # 7. Subdivision en 2 sous-cercles
    offset = (car.car_height // 4)
    dx = offset * math.cos(theta_rad)
    dy = offset * math.sin(theta_rad)

    center1 = (x, y - 2*dy)
    center2 = (x, y)

    radius = math.sqrt((car.car_width / 2) ** 2 + (car.car_height / 4) ** 2)

    if visualize:
        # Afficher les 2 sous-cercles
        ax.add_patch(patches.Circle(center1, radius=radius, edgecolor='orange', fill=False, linestyle='--'))
        ax.add_patch(patches.Circle(center2, radius=radius, edgecolor='orange', fill=False, linestyle='--'))
        ax.plot(center1[0], center1[1], 'o', color='orange', markersize=3)
        ax.plot(center2[0], center2[1], 'o', color='orange', markersize=3)
        plt.show()

    # 8. Appel récursif
    return (is_valid_circle(center1[0], center1[1], theta, car, map_img, depth + 1, max_depth, visualize) and
            is_valid_circle(center2[0], center2[1], theta, car, map_img, depth + 1, max_depth, visualize))

    














# Charger la carte
map_img = Image.open('mapHyperLittle.png').convert('RGBA')

#Coordonnées de la voiture
x = 50
y = 50
theta = 0


# Créer la voiture
car = Car(x, y, theta)



Car_simulation()




#show_car_with_circle(x, y, theta, car, map_img)
#show_car(x, y, theta, car, map_img, show_circle=False)





#result = is_valid(x, y, theta, car, map_img)
#print("Libre" if result else "Collision")

#result = is_valid_circle(x, y, theta, car, map_img)
#print("Libre" if result else "Collision")

## Version 3

Code avec OpenCV

### Archives

In [ ]:
# 1 Check Collision
def check_collision(img, x, y, theta):
    """Vérifie si la voiture entre en collision avec des zones non libres."""
    center_x = x + (CAR_LENGTH / 2) * math.cos(theta)
    center_y = y + (CAR_LENGTH / 2) * math.sin(theta)
    #cv2.circle(img, (int(center_x), int(center_y)), 3, (255, 0, 0), -1)  # Bleu BGR

    rect = ((center_x, center_y), (CAR_LENGTH, CAR_WIDTH), -np.degrees(theta))
    box = cv2.boxPoints(rect)
    box = np.int32(box)

    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    cv2.drawContours(mask, [box], 0, 255, -1)

    # Isoler la zone occupée par la voiture
    car_area = cv2.bitwise_and(img, img, mask=mask)

    # Les zones libres sont très claires (gris clair)
    # On considère tout pixel trop sombre ou coloré comme obstacle
    lower_free = np.array([180, 180, 180])
    upper_free = np.array([255, 255, 255])
    free_mask = cv2.inRange(car_area, lower_free, upper_free)

    # Compter les pixels qui NE sont PAS libres
    total_pixels = cv2.countNonZero(mask)
    free_pixels = cv2.countNonZero(free_mask)
    obstacle_pixels = total_pixels - free_pixels

    return obstacle_pixels > 5  # seuil de tolérance


# 2 Check Collision
def check_circular_collision(img, x, y, theta):
    """Vérifie une collision suspectée avec un grand cercle, puis confirme avec deux petits cercles."""

    Suspected_collision = False
    Collision = False

    # ---- Paramètres ----
    radius = int(math.hypot(CAR_LENGTH, CAR_WIDTH) / 2) + 2 # A modifier
    radius_small = radius // 2

    # ---- Couleurs libres (route, sol) ----
    lower_free = np.array([220, 220, 220])
    upper_free = np.array([255, 255, 255])

    # ---- Centre du grand cercle (avant de la voiture) ----
    center_x = int(x + (CAR_LENGTH / 4) * math.cos(theta))
    center_y = int(y - (CAR_LENGTH / 4) * math.sin(theta))

    # ---- Masque pour le grand cercle ----
    mask_big = np.zeros(img.shape[:2], dtype=np.uint8)
    cv2.circle(mask_big, (center_x, center_y), radius, 255, -1)

    circle_area_big = cv2.bitwise_and(img, img, mask=mask_big)
    free_mask_big = cv2.inRange(circle_area_big, lower_free, upper_free)

    total_pixels_big = cv2.countNonZero(mask_big)
    free_pixels_big = cv2.countNonZero(free_mask_big)
    obstacle_pixels_big = total_pixels_big - free_pixels_big

    print(obstacle_pixels_big)

    if obstacle_pixels_big > 0:
        #print("⚠️ Collision suspectée")
        Suspected_collision = False

        # ---- Vérification avec deux petits cercles ----
        center_x_1 = int(center_x - radius_small* math.cos(theta))
        center_x_2 = int(center_x + radius_small* math.cos(theta))
        center_y_1 = int(center_y + radius_small* math.sin(theta))
        center_y_2 = int(center_y - radius_small* math.sin(theta))

        # Petit cercle gauche
        mask1 = np.zeros(img.shape[:2], dtype=np.uint8)
        cv2.circle(mask1, (center_x_1, center_y_1), radius_small, 255, -1)
        circle1 = cv2.bitwise_and(img, img, mask=mask1)
        free1 = cv2.inRange(circle1, lower_free, upper_free)
        obstacle1 = cv2.countNonZero(mask1) - cv2.countNonZero(free1)

        # Petit cercle droit
        mask2 = np.zeros(img.shape[:2], dtype=np.uint8)
        cv2.circle(mask2, (center_x_2, center_y_2), radius_small, 255, -1)
        circle2 = cv2.bitwise_and(img, img, mask=mask2)
        free2 = cv2.inRange(circle2, lower_free, upper_free)
        obstacle2 = cv2.countNonZero(mask2) - cv2.countNonZero(free2)

        if obstacle1 > 0 or obstacle2 > 0:
            #print("❌ Collision détectée")
            Collision = True

    return Suspected_collision, Collision


# 3 Check Collision
def check_circular_collision_distance_1_circle(dist_map, x, y, theta):
    """Détecte une collision basée sur la distance à l'obstacle à un point précis devant la voiture."""

    Suspected_collision = False
    Collision = False  # Pas utilisée ici mais conservée pour compatibilité

    # ---- Paramètres ----
    radius = int(math.hypot(CAR_LENGTH, CAR_WIDTH) / 2) + 2
    distance_threshold = radius

    # ---- Centre du cercle (avant de la voiture) ----
    center_x = int(x + (CAR_LENGTH / 4) * math.cos(theta))
    center_y = int(y - (CAR_LENGTH / 4) * math.sin(theta))

    # ---- Sécurité pour rester dans les dimensions de l'image ----
    h, w = dist_map.shape[:2]
    if not (0 <= center_x < w and 0 <= center_y < h):
        return Suspected_collision, Collision  # Hors image

    # ---- Vérification de la distance à l'obstacle ----
    distance_at_center = dist_map[center_y, center_x]

    # Si la valeur retournée est un vecteur (multi-canaux), on prend le premier canal ou la moyenne
    if isinstance(distance_at_center, np.ndarray):
        distance_at_center = distance_at_center[0]  # ou np.mean(...) si souhaité

    if distance_at_center < distance_threshold:
        Suspected_collision = True

    return Suspected_collision, Collision


# Pour déterminer les différentes couleurs dans l'images
def list_unique_colors(img):
    """
    Affiche la liste des couleurs uniques (pixels) présentes dans une image BGR.
    Renvoie aussi la liste des couleurs.
    """
    # Redimensionne l'image en un tableau de pixels (N, 3)
    pixels = img.reshape(-1, img.shape[2])

    # Supprime les doublons
    unique_colors = np.unique(pixels, axis=0)

    print(f"Nombre de couleurs uniques : {len(unique_colors)}")
    for i, color in enumerate(unique_colors):
        print(f"{i}: {tuple(color)}")  # BGR format

    return unique_colors
#unique_colors = list_unique_colors(map_img)


In [3]:
import numpy as np

print(45*np.pi/180)
print(np.pi/4)


0.7853981633974483
0.7853981633974483
